In [1]:
using AMLET, RDST, Distributions, ForwardDiff, GERALDINE

In [2]:
ngamma = 8

8

In [3]:
struct DummyBatch <: Batch
    IND::Array{LM_Individual}
    rng::MRG32k3a
end

import Base.iterate
function iterate(db::DummyBatch)
    state = 1
    reset_stream!(db.rng)
    if state <= length(db.IND)
        return MLM_Individual(db.IND[state], db.rng, ngamma), state+1
    else
        return nothing
    end
end
function iterate(db::DummyBatch, state::Int = 1)
    if state <= length(db.IND)
        next_substream!(db.rng)
        return MLM_Individual(db.IND[state], db.rng, ngamma), state+1
    else
        return nothing
    end
end

iterate (generic function with 271 methods)

In [4]:
gen = MRG32k3aGen([125,8765423,4546,6434,645,76465])
betaGen = next_stream(gen)
monteCarlosGen = next_stream(gen)

Full state of MRG32k3a generator:
Cg = [2459988903, 764778996, 4187205638, 4089700229, 517468743, 2169295582]
Bg = [2459988903, 764778996, 4187205638, 4089700229, 517468743, 2169295582]
Ig = [2459988903, 764778996, 4187205638, 4089700229, 517468743, 2169295582]

In [5]:
function beta(θ::Vector, rng)
    N = Normal.(θ[1:4], θ[5:8])
    return [rand(rng, N[1]), rand(rng, N[2]), rand(rng, N[3]), rand(rng, N[4])]
end

beta (generic function with 1 method)

In [6]:
gum = Gumbel()

Gumbel{Float64}(μ=0.0, θ=1.0)

In [7]:
function genIND(dists::Array{T, 1}, θ::Vector, m::Int) where T<: Distribution
    β = beta(θ, betaGen)
    N = length(dists)
    param = Array{Float64, 2}(undef, 0, N)
    for _ in 1:m
        param = [param ; (rand.(dists))']
    end
    best = argmax(param*β + rand(gum, m))
    return LM_Individual(param, best, 1)
end

genIND (generic function with 1 method)

In [8]:
θstar = [5,6,7,8, 1,1,1,1]

8-element Array{Int64,1}:
 5
 6
 7
 8
 1
 1
 1
 1

In [9]:
dists = [Normal(2, 4) for _ in 1:4]
genIND(dists, θstar, 5)

LM_Individual{Array{Float64,2}}([2.717131814896976 -1.61050775820401 0.8323190344224061 2.9252797069751657; -1.1900984511465955 6.01037753051534 3.6783536089786733 6.066558631417099; … ; 6.454821058292465 -1.0203000572869518 6.503579814144955 5.685134647368919; 0.9559566681983296 -0.5446896587829566 -0.917421201949403 3.9855305977089053], 2, 1)

In [10]:
db = DummyBatch([genIND(dists, θstar, 5) for _ in 1:1_000], monteCarlosGen);

In [11]:
mlm = MLM(db);

In [12]:
#linear utilities of mixed logit model where each parameters of beta are assumed to follow an 
#independant exponnential Distribution beta

function EU(θ::Vector, X::Matrix, rng::Rigged{N}) where N
    β = beta(θ, rng)
    reset_stream!(rng)
    return X*β
end

function EU_i(θ::Vector, X::Matrix, i::Int64, rng::Rigged{N}) where N
    β = beta(θ, rng)
    reset_stream!(rng)
    return X[i, :]'*β
end

function ∇EU(θ::Vector, X::Matrix, rng::Rigged{N}) where N
    return vcat([∇EU_i(θ, X, i, rng) for i in 1:size(X, 1)]...)
end

function ∇EU_i(θ::Vector, X::Matrix, i::Int64, rng::Rigged{N}) where N
    function tmp(x::Vector)
        return EU_i(x, X, i, rng)
    end
    return ForwardDiff.gradient(tmp, θ)
end

#uti = AMLET.LinearUtilitie(EU, ∇EU, EU_i, ∇EU_i)


∇EU_i (generic function with 1 method)

In [13]:
#linear utilities of mixed logit model where each parameters of beta are assumed to follow an 
#independant exponnential Distribution beta

function EU(θ::Vector, X::Matrix, rng::Rigged{N}) where N
    β = θ[1:4] + θ[5:8] .* [rand(rng, Normal(0, 1)) for _ in 1:4]
    reset_substream!(rng)
    return X*β
end

function EU_i(θ::Vector, X::Matrix, i::Int64, rng::Rigged{N}) where N
    β = θ[1:4] + θ[5:8] .* [rand(rng, Normal(0, 1)) for _ in 1:4]
    reset_substream!(rng)
    return X[i, :]'*β
end

function ∇EU(θ::Vector, X::Matrix, rng::Rigged{N}) where N
    return vcat([∇EU_i(θ, X, i, rng) for i in 1:size(X, 1)]...)
end

function ∇EU_i(θ::Vector, X::Matrix, i::Int64, rng::Rigged{N}) where N
    function tmp(x::Vector)
        return EU_i(x, X, i, rng)
    end
    return ForwardDiff.gradient(tmp, θ)
end

uti = AMLET.LinearUtilitie(EU, ∇EU, EU_i, ∇EU_i)


AMLET.LinearUtilitie(EU, ∇EU, EU_i, ∇EU_i)

In [14]:
complete_Model!(mlm, uti, 20)

(::AMLET.var"#∇F!#17"{DummyBatch,AMLET.LinearUtilitie,Int64}) (generic function with 2 methods)

In [15]:
mlm.f(ones(8))

0.46688960856960326

In [16]:
mlm.f(ones(8))

0.46688960856960326

In [17]:
grad1 = zeros(8)
grad2 = zeros(8)

8-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [18]:
mlm.∇f!(ones(8), grad1)
println(grad1)
mlm.∇f!(ones(8), grad2)

[0.02483587707561422, -0.046354045319964614, -0.09945128224612032, -0.13753504132160044, 0.08611999945013388, 0.06675504952017128, 0.05537264469870458, 0.032402618624142204]


8-element Array{Float64,1}:
  0.02483587707561422 
 -0.046354045319964614
 -0.09945128224612032 
 -0.13753504132160044 
  0.08611999945013388 
  0.06675504952017128 
  0.05537264469870458 
  0.032402618624142204

In [19]:
grad1 == grad2

true

# Using the basic trust region (BTR) algorithm with BFGS approximation

In [20]:
using GERALDINE

In [21]:
 θstar

8-element Array{Int64,1}:
 5
 6
 7
 8
 1
 1
 1
 1

In [ ]:
x = OPTIM_BFGS(mlm.f, mlm.∇f!, x0 = 1.0*θstar, nmax = 200, verbose = true)

[5.006, 6.0, 7.002, 7.985, 1.039, 1.004, 1.002, 1.01]
[5.023, 5.992, 7.028, 7.906, 1.24, 1.021, 1.009, 1.061]
[5.013, 5.99, 7.05, 7.871, 1.33, 1.026, 1.009, 1.084]
[4.99, 5.991, 7.072, 7.851, 1.38, 1.025, 1.007, 1.098]
[4.863, 6.002, 7.174, 7.773, 1.589, 1.008, 0.991, 1.158]
[4.8, 6.011, 7.21, 7.756, 1.656, 0.992, 0.977, 1.175]
[4.694, 6.032, 7.25, 7.763, 1.701, 0.952, 0.938, 1.183]
[4.667, 6.045, 7.227, 7.825, 1.594, 0.921, 0.91, 1.148]
[4.636, 6.056, 7.217, 7.868, 1.53, 0.887, 0.876, 1.12]
[4.622, 6.061, 7.219, 7.878, 1.523, 0.868, 0.855, 1.108]
[4.591, 6.084, 7.228, 7.906, 1.514, 0.762, 0.727, 1.046]
[4.626, 6.087, 7.229, 7.892, 1.539, 0.717, 0.665, 1.018]
[4.675, 6.073, 7.233, 7.854, 1.573, 0.746, 0.694, 1.035]
[4.68, 6.074, 7.24, 7.847, 1.57, 0.738, 0.685, 1.033]
[4.696, 6.082, 7.31, 7.784, 1.529, 0.688, 0.626, 1.035]
[4.683, 6.083, 7.306, 7.793, 1.523, 0.695, 0.637, 1.042]
[4.677, 6.082, 7.309, 7.792, 1.521, 0.697, 0.643, 1.053]
[4.676, 6.081, 7.312, 7.789, 1.52, 0.695, 0.644, 1.

In [ ]:
grad = zeros(8)

In [ ]:
mlm.∇f!(x[1], grad)

In [ ]:
[1,2,3][4]